# Using neural classification
As has been proven by [Wang (2017)](https://arxiv.org/abs/1705.00648), neural classifiers carry better results than non-neural classifiers when detecting fake news. However, it is unknown how well neural networks classify fake news when using previously mentioned text embeddings. 
In this notebook, the second research question will be answered: *how well do neural network architecture classify fake news compared to non-neural classification algorithms?*

<hr>

## On the usage of neural networks
Literature on CNNs and Bi-LSTMs


In [12]:
# General imports
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from sklearn.preprocessing import MultiLabelBinarizer
import keras
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Reshape, Conv2D, MaxPooling2D, Flatten

# Set offline mode for plotly
init_notebook_mode(connected = True)

# The DataLoader class gives access to pretrained vectors from the Liar dataset
from data_loader import DataLoader
data = DataLoader()

In [2]:
general = data.get_dfs()

# Recode labels from 6 to 3
def recode(label):
    if label == 'false' or label == 'pants-fire' or label == 'barely-true':
        return 0
    elif label == 'true' or label == 'mostly-true':
        return 2
    elif label == 'half-true':
        return 1

for dataset in general.keys():
    general[dataset]['label'] = general[dataset]['label'].apply(lambda label: recode(label))

<hr>

## Bidirectional LSTMs

In [3]:
# Get max-pooled BERT embeddings from RQ1
bert = data.get_bert()

def max_pool(statement):
    if len(statement) > 1:
        return [row.max() for row in np.transpose([[token_row.max() for token_row in np.transpose(np.array(sentence))] for sentence in statement])]
    else:
        return [token_row.max() for token_row in np.transpose(statement[0])]

max_pooled_bert = {
    dataset: pd.DataFrame(list(bert[dataset].statement.apply(lambda statement: max_pool(statement)).values))
    for dataset in bert.keys()
}

In [4]:
def get_bilstm_score(X_train, X_test, X_validation, y_train = general['train']['label'], y_test = general['test']['label'], y_validation = general['validation']['label']):
    # Rearrange data types    
    params = locals().copy()    
    inputs = {
        dataset: np.array(params[dataset])
        for dataset in params.keys()
    }
    
    # Reshape datasets
    for dataset in inputs.keys():
        if dataset[0:1] == 'X':
            inputs[dataset] = np.reshape(inputs[dataset], (inputs[dataset].shape[0], inputs[dataset].shape[1], 1))
        elif dataset[0:1] == 'y':
            inputs[dataset] = np_utils.to_categorical(np.array(inputs[dataset]), 3)
    
    # Set model parameters
    epochs = 5
    batch_size = 32
    input_shape = X_train.shape

    # Create the model
    model = Sequential()
    model.add(Bidirectional(LSTM(64, input_shape = input_shape)))
    model.add(Dropout(0.8))
    model.add(Dense(3, activation = 'softmax'))
    model.compile('sgd', 'categorical_crossentropy', metrics = ['accuracy']) 
    
    # Fit the training set over the model and correct on the validation set
    model.fit(inputs['X_train'], inputs['y_train'],
            batch_size = batch_size,
            epochs = epochs,
            validation_data = (inputs['X_validation'], inputs['y_validation']))
    
    # Get score over the test set
    score, acc = model.evaluate(inputs['X_test'], inputs['y_test'])
    
    return acc

In [5]:
get_bilstm_score(max_pooled_bert['train'], max_pooled_bert['test'], max_pooled_bert['validation'])

Instructions for updating:
Colocations handled automatically by placer.
2019-05-21 12:40:37,047 From /Users/martijn/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
2019-05-21 12:40:37,448 From /Users/martijn/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
2019-05-21 12:40:37,537 From /Users/martijn/anaconda3/lib/python3.6/site-packages/tensorflo

0.43715415052745654

<hr>

## Convolutional neural networks

In [31]:
def get_cnn_score(X_train, X_test, X_validation, y_train = general['train']['label'], y_test = general['test']['label'], y_validation = general['validation']['label']):
    # Rearrange data types    
    params = locals().copy()    
    inputs = {
        dataset: np.array(params[dataset])
        for dataset in params.keys()
    }
    
    # Reshape datasets
    for dataset in inputs.keys():
        if dataset[0:1] == 'X':
            inputs[dataset] = np.reshape(inputs[dataset], (inputs[dataset].shape[0], inputs[dataset].shape[1], 1))
        elif dataset[0:1] == 'y':
            inputs[dataset] = np_utils.to_categorical(np.array(inputs[dataset]), 3)
    
    # Set model parameters
    epochs = 5
    batch_size = 32
    input_shape =  inputs['X_train'].shape
    print(inputs['X_train'].shape)
    # Create the model
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape = input_shape))
    #model.add(Conv2D(64, (3, 3), activation='relu'))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))
    #model.add(Flatten())
    #model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation = 'softmax'))
    model.compile('sgd', 'categorical_crossentropy', metrics = ['accuracy']) 
    
    # Fit the training set over the model and correct on the validation set
    model.fit(inputs['X_train'], inputs['y_train'],
            batch_size = batch_size,
            epochs = epochs,
            validation_data = (inputs['X_validation'], inputs['y_validation']))
    
    # Get score over the test set
    score, acc = model.evaluate(inputs['X_test'], inputs['y_test'])
    
    return acc

In [32]:
get_cnn_score(max_pooled_bert['train'], max_pooled_bert['test'], max_pooled_bert['validation'])

(10235, 3072, 1)


ValueError: Error when checking input: expected conv2d_16_input to have 4 dimensions, but got array with shape (10235, 3072, 1)

<hr>

### References

```
@article{DBLP:journals/corr/Wang17j,
  author    = {William Yang Wang},
  title     = {"Liar, Liar Pants on Fire": {A} New Benchmark Dataset for Fake News
               Detection},
  journal   = {CoRR},
  volume    = {abs/1705.00648},
  year      = {2017},
  url       = {http://arxiv.org/abs/1705.00648},
  archivePrefix = {arXiv},
  eprint    = {1705.00648},
  timestamp = {Mon, 13 Aug 2018 16:48:58 +0200},
  biburl    = {https://dblp.org/rec/bib/journals/corr/Wang17j},
  bibsource = {dblp computer science bibliography, https://dblp.org}
}
```